In [1]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Step 1. Load Dataset
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=False)

# Step 2. Make Dataset Iterable
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
# Step 3. Create Model Class
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [3]:
epochs = 2
input_dim = 784
output_dim = 10
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# output_dim = 1
lr_rate = 0.01

# Step 4. Instantiate Model Class
model = LogisticRegression(input_dim, output_dim)
# Step 5. Instantiate Loss Class
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# criterion = torch.nn.MSELoss()
# Step 6. Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [4]:
# Step 7. Train Model
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        # [test] 만일 MSE을 LOSS 함수로 쓴다면???
        # labels = labels.type(torch.FloatTensor)
        # outputs = outputs.type(torch.FloatTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%100==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

Iteration: 100. Loss: 1.5278689861297607. Accuracy: 75.83000183105469.
Iteration: 200. Loss: 1.1347965002059937. Accuracy: 81.0999984741211.
Iteration: 300. Loss: 1.0471291542053223. Accuracy: 82.70999908447266.
Iteration: 400. Loss: 0.9169452786445618. Accuracy: 83.91000366210938.
Iteration: 500. Loss: 0.8972954750061035. Accuracy: 85.0199966430664.
Iteration: 600. Loss: 0.9665325284004211. Accuracy: 85.41999816894531.
Iteration: 700. Loss: 0.6086663007736206. Accuracy: 86.05000305175781.
Iteration: 800. Loss: 0.6255840063095093. Accuracy: 86.37000274658203.
Iteration: 900. Loss: 0.602271318435669. Accuracy: 86.9000015258789.
Iteration: 1000. Loss: 0.6010639071464539. Accuracy: 86.95999908447266.
Iteration: 1100. Loss: 0.6313837170600891. Accuracy: 87.20999908447266.
Iteration: 1200. Loss: 0.6083102822303772. Accuracy: 87.51000213623047.
